# Quesiton 1 - Data Loading and Cleaning

[19882,
 7697,
 14041,
 31273,
 31960,
 12913,
 22505,
 1677,
 32226,
 1147,
 712,
 11570,
 21662,
 179,
 28179,
 3802,
 18159,
 25238,
 17066,
 2893,
 25219,
 25141,
 12738,
 14097,
 6158,
 7097,
 1015,
 31498,
 24274,
 22319,
 19654,
 31498,
 22599,
 20384,
 18159,
 28151,
 22505,
 24188,
 4203,
 19937,
 23381,
 33169,
 5246,
 12758,
 2668,
 22505,
 13003,
 14219,
 6300,
 27119,
 3054,
 19937,
 23381,
 33169,
 12892,
 29891,
 21382,
 613,
 17066,
 13003,
 1147,
 26140,
 31498,
 9065,
 31273,
 31960,
 21058,
 712,
 18108,
 5009,
 32818,
 11570,
 21662,
 179,
 28179,
 3802,
 18159,
 25238,
 22505,
 5064,
 17066,
 2893,
 13265,
 19937,
 23381,
 33169,
 5702,
 7697,
 17345,
 1144,
 16001,
 31273,
 31976,
 13003,
 16846,
 32014,
 1677,
 32226,
 1147,
 22505,
 14650,
 32226,
 27238,
 21382,
 6300,
 21366,
 6300,
 22505,
 28179,
 13064,
 20022,
 20342,
 23145,
 32226,
 28963,
 29463,
 19937,
 23381,
 33169,
 18862,
 11281,
 32226,
 23359,
 4868,
 47,
 13265,
 27372,
 28599,
 16312,
 22352,


In [1]:
#get data to list
def turnToList(string):  
    f = open(string, 'r')
    x = f.readlines()
    #print(x)
    articleVocab = x[0]
    ls = articleVocab.split("', '")
    return ls

In [2]:
training_data = turnToList("Data/group1.train.txt")

In [3]:
import math

def makeBatches(data,batch_size):
    cuts = len(data)/batch_size
    cuts = math.ceil(cuts)
    final_list = [data[i:i+cuts] for i in range(0,len(data),cuts)]
    return final_list
    

In [4]:
bitches = makeBatches(training_data, 30)

In [5]:
print(len(bitches[28]))

156473


In [6]:
def sliding_windows(batch,n):
    window = [batch[i:i+n] for i in range(0,len(batch))]
    return window

In [7]:
batch1 = bitches[1]
w1 = sliding_windows(batch1,5)
batch2 = bitches[2]
w2 = sliding_windows(batch2,5)

In [9]:
#question 1 output
training_sequences = {}
training_sequences['Batch 1'] = [w1[i] for i in range(0,3)]
training_sequences['Batch 2'] = [w2[i] for i in range(0,3)]

In [10]:
print(training_sequences)

{'Batch 1': [['the', 'rich', '.', '</s>', '<s>'], ['rich', '.', '</s>', '<s>', 'By'], ['.', '</s>', '<s>', 'By', '2018']], 'Batch 2': [['to', 'strike', 'Soleimani', ',', 'who'], ['strike', 'Soleimani', ',', 'who', 'was'], ['Soleimani', ',', 'who', 'was', 'visiting']]}


## Question 2 - Bengio Model

In [77]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import random
import math
import time

In [78]:
#bring in pickle file for vocab dict
import pickle

file = open('Data/trainingDictPickle1_group1.pkl','rb')
vocab_dict = pickle.load(file)
file.close()

In [79]:
class Parameters():
    def __init__(self, vocab_dict):
        self.window_size = 5
        self.num_batches = 30

        self.vocab_size = len(vocab_dict)

        self.hidden_units = 50
        self.embeddings_dim = 60
        self.num_epochs = 20

        self.learning_rate = 0.5

        self.gpu_mem = 0.25

        self.tf_precision = tf.float32
        self.np_precision = np.float32

        self.init_scale = 0.5
        self.max_grad = 10.0

In [80]:
class Model(object):
    def __init__(self, params=Parameters(vocab_dict)):
        #initialize input and output variables
        self._y = tf.placeholder(dtype=params.tf_precision,shape=(params.num_batches, params.vocab_size),name="Y")
        self._x = tf.placeholder(tf.int32,shape=(params.num_batches, params.window_size),name="X")
        #word features/embeddings
        self.C = tf.Variable(tf.truncated_normal(shape=(params.vocab_size, params.embeddings_dim),mean=-1,stddev=-1),dtype=params.tf_precision,name="C")
        #projection to output weight
        self.W = tf.Variable(tf.random_normal(shape=(params.vocab_size, (params.window_size) * params.embeddings_dim)),dtype=params.tf_precision,name="W")
        #hidden layer weight and bias
        self.H = tf.Variable(tf.random_normal(shape=(params.hidden_units, (params.window_size) * params.embeddings_dim)),dtype=params.tf_precision,name="H")
        self.d = tf.Variable(tf.random_normal(shape=(params.hidden_units,)),dtype=params.tf_precision,name="d")
        #hidden layer to output weight and bias
        self.U = tf.Variable(tf.random_normal((params.vocab_size, params.hidden_units)),name="U",dtype=params.tf_precision)
        self.b = tf.Variable(tf.random_normal(shape=(params.vocab_size, )),dtype=params.tf_precision,name="b")
        #do calculations
        # y = b + Wx + Utanh(d + Hx)
        # x = (C(w(t-1)), C(w(t-2), ..., C(w(t-n+1))), n == window_size
        with tf.name_scope('Projection_Layer'):
            x  = tf.nn.embedding_lookup(self.C, self._x) # (batch_size, window_size-1, emb_dim)
            x  = tf.reshape(x, shape=(params.num_batches, (params.window_size) * params.embeddings_dim))
        with tf.name_scope('Hidden_Layer'):
            Hx = tf.matmul(x, tf.transpose(self.H)) # (batch_size, hidden_size)
            o  = tf.add(self.d, Hx) # (batch_size, hidden_size)
            a  = tf.nn.tanh(o)  # (batch_size, hidden_size)
        with tf.name_scope('Output_Layer'):
            Ua = tf.matmul(a, tf.transpose(self.U)) # (batch_size, vocab_size)
            Wx = tf.matmul(x, tf.transpose(self.W)) # (batch_size, vocab_size)
            y_hat  = tf.nn.softmax(tf.clip_by_value(tf.add(self.b, tf.add(Wx, Ua)), 0.0, 10)) # (batch_size, vocab_size)
    #2 different ways to minimize loss/cost
        with tf.name_scope('Cost'):
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self._y,logits=y_hat))
            self.perplexity = tf.exp(self.cost)
        self.optimizer = tf.train.GradientDescentOptimizer(params.learning_rate).minimize(self.cost)
    #loss function defined from github repo
    '''
        with tf.name_scope('Loss'):
            onehot_tgt = tf.one_hot(tf.squeeze(self._y), params.vocab_size)  # (batch_size, vocab_size)
            loss = -1 * tf.reduce_mean(tf.reduce_sum(tf.log(y) * onehot_tgt, 1)) # 乘 -1 -> maximize loss
            print(loss)
        
        optimizer = tf.train.AdagradOptimizer(params.learning_rate).minimize(loss)
        '''


In [105]:
def get_Windows(batch, window_index, params):
    x_window = batch[window_index:window_index + params.window_size]
    y_targets = np.zeros(params.vocab_size)
    #update y_targets according to each new predicted word in the sliding window
    y_targets[batch[window_index + params.window_size]] = 1
    return x_window, y_targets

In [106]:
def run_model(model,params,integer_batches):
    #initialize and do GPU stuff
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
    
    #get lists to hold perplexity and cost graphs
    perplexity_history = []
    cost_history = []

    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=True)) as sess:
        initializer = tf.global_variables_initializer()
        initializer.run()
        step = 0
        #iterate through epochs
        for epoch in range(params.num_epochs):
            window_index = 0
            #feed in each sliding context window
            while window_index < len(integer_batches[0]) - params.window_size - 1:
                x_batch = []
                y_batch = []
                for batch in integer_batches:
                    x_window, y_targets = get_Windows(batch,window_index,params)
                    x_batch.append(x_window)
                    y_batch.append(y_targets)
                #run iteration of batches in the model
                cost, perplexity, deez_nutz = sess.run([model.cost,model.perplexity, model.optimizer], feed_dict = {model._x:x_batch, model._y:y_batch})
#                 if step % 1000 == 0:
                perplexity_history.append(perplexity)
                cost_history.append(cost)
                window_index += 1
                step += 1
    return perplexity_history, cost_history

## Brown Corpus

In [107]:
file = open('Brown Corpus/bc_unbatched.pkl','rb')
brown_batches = pickle.load(file)
file.close()

In [108]:
brown_vocab = set([tok for tok in brown_batches])

In [109]:
# create train, validation, test
brown_train = brown_batches[:800000]
brown_validation = brown_batches[800000:1000000]
brown_test = brown_batches[1000000:]

In [110]:
def dict_maker(lst):
    a = {val : idx + 1 for idx, val in enumerate(lst)}
    return a

In [111]:
brown_vocab_dicks_in_my_mouth = dict_maker(brown_vocab)

In [112]:
brown_train_int = [brown_vocab_dicks_in_my_mouth[tok] for tok in brown_train]
brown_val_int = [brown_vocab_dicks_in_my_mouth[tok] for tok in brown_validation]
brown_test_int = [brown_vocab_dicks_in_my_mouth[tok] for tok in brown_test]

In [113]:
# train on brown
brown_train_batches_int = makeBatches(brown_train_int,30)
brown_val_batches_int = makeBatches(brown_val_int,30)
brown_test_batches_int = makeBatches(brown_test_int,30)

In [114]:
brown_params = Parameters(brown_vocab_dicks_in_my_mouth)
brown_model = Model(params=brown_params)

In [ ]:
%time
brown_train_perplexity_history, brown_train_cost_history = run_model(brown_model,brown_params,brown_train_batches_int)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 9.3 µs
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

